# Assess prediction

## Dimensions to consider

- year
- season
- hourly

- daily count vs hourly count

## Variable to consider

- weather


In [ ]:
import math

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

In [ ]:
path = r"C:\Users\amedee.roy\OneDrive - FRANCE ENERGIES MARINES\SCRIPTS\defile-migration-forecast\logs\eval\runs\2024-07-11_16-29-02\test_pred.nc"
data = xr.open_dataset(path)

## Global metric


In [ ]:
true_count = data.masked_total_counts.values
pred_count = data.estimated_masked_total_counts.values

plt.hist(true_count, label="True count", alpha=0.5)
plt.hist(pred_count, label="Predicted count", alpha=0.5)
plt.xlabel("Count (log-scale)")
plt.ylabel("Histogram")
plt.legend()
plt.show()

In [ ]:
x = np.linspace(min(true_count), max(true_count), 100)
coef = np.polyfit(true_count, pred_count, 5)
y = np.polyval(coef, x)

plt.scatter(true_count, pred_count, c="black", s=5, alpha=0.4)
plt.plot(x, y, c="red")
plt.plot(x, x, "--", c="black")
plt.xlabel("True count (log-scale)")
plt.ylabel("Predicted count (log-scale)")
plt.show()

## Timeseries


In [ ]:
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(4, 4, figsize=(12, 8), tight_layout=True)
ax = ax.flatten()
for i, k in enumerate(np.random.randint(0, len(data.date), size=16)):
    subs = data.isel(date=k)

    ax[i].plot(np.arange(0, 24), subs.estimated_hourly_counts)
    for k, m in enumerate(subs.mask.values):
        if m == 1:
            ax[i].add_patch(Rectangle((k, 0), m, 10, color="yellow"))
            obs = subs.masked_total_counts.item() / subs.mask.sum().item()
            ax[i].plot([k, k + 1], [obs, obs], c="tab:red")

    ax[i].set_ylim(0, max(subs.estimated_hourly_counts.max(), obs) + 0.1)
    ax[i].set_xlabel("hours")
    ax[i].set_ylabel("Bird counts (log10)")
# plt.savefig(f'{self.plotdir}/plt_timeseries.jpg')

In [ ]:
data = data.assign_coords(doy=data.date.dt.dayofyear)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
data.groupby("doy").sum().masked_total_counts.plot(ax=ax, label="True")
data.groupby("doy").sum().estimated_masked_total_counts.plot(ax=ax, label="Prediction")
plt.legend()
plt.show()